In [1]:
# load dataset csv
import pandas as pd
n_samples_omit = 48

concat_data = pd.DataFrame()

for survey_idx in range(0, n_samples_omit):
    SURVEY_NUMBER = survey_idx + 1
    if SURVEY_NUMBER <= 40:
        number_of_questions = 20
    else:
        number_of_questions = 19
    omit_csv_filename = "./forms/dataset{:03}.csv".format(SURVEY_NUMBER)
    omit_CSV = pd.read_csv(omit_csv_filename)
    survey_content = omit_CSV[['filename', 'turn', 'similarity']]
    response_csv_filename = "./forms/responses" + str(SURVEY_NUMBER) + ".csv"
    response_CSV = pd.read_csv(response_csv_filename)
    number_of_responses = [len(response_CSV)] * number_of_questions
    nor_pd = pd.DataFrame(number_of_responses, columns=['nor'])
    response_content = response_CSV.iloc[:, 2:(2 + number_of_questions)]
    response_content = response_content.transpose()
    columns = ['res' + str(i) for i in range(len(response_CSV))]
    response_content = response_content.set_axis(columns, axis=1, copy=False)
    rows = range(number_of_questions)
    response_content = response_content.set_axis(rows, axis=0, copy=False)
    combined_data = pd.concat([survey_content, nor_pd, response_content], axis=1)
    concat_data = pd.concat([concat_data, combined_data])

concat_data.info()
print(concat_data.describe())


<class 'pandas.core.frame.DataFrame'>
Index: 952 entries, 0 to 18
Data columns (total 17 columns):
 #   Column      Non-Null Count  Dtype  
---  ------      --------------  -----  
 0   filename    952 non-null    object 
 1   turn        952 non-null    int64  
 2   similarity  952 non-null    float64
 3   nor         952 non-null    int64  
 4   res0        952 non-null    object 
 5   res1        952 non-null    object 
 6   res2        952 non-null    object 
 7   res3        952 non-null    object 
 8   res4        952 non-null    object 
 9   res5        952 non-null    object 
 10  res6        952 non-null    object 
 11  res7        952 non-null    object 
 12  res8        932 non-null    object 
 13  res9        932 non-null    object 
 14  res10       415 non-null    object 
 15  res11       217 non-null    object 
 16  res12       40 non-null     object 
dtypes: float64(1), int64(2), object(14)
memory usage: 133.9+ KB
             turn  similarity         nor
count  952.0000

In [5]:
# Create CSV for dataset evaluation
import pandas as pd
import csv

SPEAKERS = ["A: ", "B: "]

for idx in range(0, 50):
    sample_CSV = pd.DataFrame({
        "filename": [],
        "turn": [],
        "complete_dialogue": [],
        "core_sentence": [],
    })
    sample_CSV = sample_CSV.astype({
        "filename": str,
        "turn": int,
        "complete_dialogue": str,
        "core_sentence": str
    })
    filenames = ["PP" + str(idx * 2 + 1) + ".txt", "PP" + str(idx * 2 + 2) + ".txt"]
    for filename in filenames:
        with open("./dat/" + filename) as f:
            dialogue = [l.rstrip() for l in f.readlines()]
        for turn, core_sentence in enumerate(dialogue[3:], 3):
            full_dialogue = "\n\n".join(dialogue)
            for speaker in SPEAKERS:
                core_sentence = core_sentence.removeprefix(speaker)
            eval_base = pd.DataFrame({
                "filename": [filename],
                "turn": [turn],
                "complete_dialogue": [full_dialogue],
                "core_sentence": [core_sentence]
            })
            eval_res = concat_data.query('filename == @filename & turn == @turn')
            eval_res = eval_res.reset_index(drop=True)
            eval_data = pd.concat([eval_base, eval_res], axis=1)
            eval_data = eval_data.loc[:, ~eval_data.columns.duplicated()]
            sample_CSV = pd.concat([sample_CSV, eval_data])

    csv_filename = "./forms/dataset{:03}eval.csv".format(idx + 1)
    sample_CSV.to_csv(csv_filename)